In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
from garminconnect import (
    Garmin,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
    GarminConnectAuthenticationError,
)
from datetime import date
import datetime
import pandas as pd
pd.options.display.max_columns = None
import json 
import pymongo
from datetime import datetime
import logging
today = date.today()

In [4]:
from configparser import ConfigParser

#Read config.ini file
config_object = ConfigParser()
config_object.read("config_dev.ini")

#Get the password
userinfo = config_object["USERINFO"]

In [5]:
"""
Initialize Garmin client with credentials
"""
try:
    client = Garmin(userinfo["email"],userinfo["password"])
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occured during Garmin Connect Client init: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occured during Garmin Connect Client init")
    quit()

In [6]:
"""
Login to Garmin Connect portal
"""
try:
    client.login()
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occured during Garmin Connect Client login: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occured during Garmin Connect Client login")
    quit()

In [7]:
"""
Get full name from profile
"""
try:
    logging.info('Garmin user : '+client.get_full_name())
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occured during Garmin Connect Client get full name: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occured during Garmin Connect Client get full name")
    quit()

In [8]:
"""
Get unit system from profile
"""
try:
    unitsystem = client.get_unit_system()
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occurred during Garmin Connect Client get unit system: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occurred during Garmin Connect Client get unit system")
    quit()
unitsystem

'metric'

In [9]:

"""
Get activities data
"""
try:
    activities = client.get_activities(0,10000) # 0=start, 1=limit
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occured during Garmin Connect Client get activities: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occured during Garmin Connect Client get activities")
    quit()
    
len(activities)

133

In [ ]:
#Create pandas dataframe from data
df_act = pd.DataFrame(activities)
df_act.head()

In [ ]:
"""
Setup Mongo Connection
"""
mongoclient = "mongodb://mongo-app:27017/"
mongodb = "activities"
mongocol = "general_tests"
mongounit = "metadata_tests"

In [ ]:
"""
Connect to Mongo DB
"""
myclient = pymongo.MongoClient(mongoclient)
mydb = myclient[mongodb]
mycol = mydb[mongocol]
mycolunit = mydb[mongounit]

# Get last activity
#last_activity = mycol.find().sort("_id").limit(1)[0]
#last_activity

In [ ]:
full_activities = df_act[['activityId','activityName','activityType','distance','duration','averageHR','startTimeLocal']]
full_activities.loc[:, 'duration min'] = full_activities['duration']/60
full_activities['activityType'] = full_activities['activityType'].apply(
    lambda x : x['typeKey'])
full_activities['startTimeLocal'] = full_activities['startTimeLocal'].apply(
    lambda x :datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
full_activities['week'] = full_activities['startTimeLocal'].apply(lambda x : x.strftime("%V"))
full_activities['date'] = full_activities['startTimeLocal'].apply(lambda x : x.date())
full_activities['date'] =  pd.to_datetime(full_activities['date']).dt.strftime('%Y-%m-%d')
full_activities.head(15)

In [ ]:



records = json.loads(full_activities.T.to_json()).values()

## handle units

In [ ]:
metadata = {
    '_id':'metadata',
    'unitsystem':unitsystem
}

mycolunit.insert_one(metadata)

In [ ]:
mycolunit.find_one()

## insert many

In [ ]:
mycol.insert_many(records)

### update

In [ ]:
for doc in records:
    info = mycol.replace_one(
        {'activityId' : doc['activityId']},
        doc,
        upsert=True
    )
    print(info)

## User stats

In [10]:
"""
Get stats and body composition data
"""
try:
    stats = client.get_stats_and_body(today.isoformat())
except (
    GarminConnectConnectionError,
    GarminConnectAuthenticationError,
    GarminConnectTooManyRequestsError,
) as err:
    print("Error occurred during Garmin Connect Client get stats and body composition: %s" % err)
    quit()
except Exception:  # pylint: disable=broad-except
    print("Unknown error occurred during Garmin Connect Client get stats and body composition")
    quit()

In [11]:
stats['_id'] = 'userProfile'
stats

{'userProfileId': 83095189,
 'totalKilocalories': 1577.0,
 'activeKilocalories': 242.0,
 'bmrKilocalories': 1335.0,
 'wellnessKilocalories': 1577.0,
 'burnedKilocalories': None,
 'consumedKilocalories': None,
 'remainingKilocalories': 1413.0,
 'totalSteps': 1226,
 'netCalorieGoal': None,
 'totalDistanceMeters': 998,
 'wellnessDistanceMeters': 998,
 'wellnessActiveKilocalories': 242.0,
 'netRemainingKilocalories': 242.0,
 'userDailySummaryId': 83095189,
 'calendarDate': '2020-07-30',
 'rule': {'typeId': 3, 'typeKey': 'subscribers'},
 'uuid': 'da9cd0b915eb46cabe63ac37983de0c2',
 'dailyStepGoal': 10000,
 'wellnessStartTimeGmt': '2020-07-29T22:00:00.0',
 'wellnessStartTimeLocal': '2020-07-30T00:00:00.0',
 'wellnessEndTimeGmt': '2020-07-30T13:41:00.0',
 'wellnessEndTimeLocal': '2020-07-30T15:41:00.0',
 'durationInMilliseconds': 56460000,
 'wellnessDescription': None,
 'highlyActiveSeconds': 977,
 'activeSeconds': 1019,
 'sedentarySeconds': 31004,
 'sleepingSeconds': 23460,
 'includesWellnes

In [ ]:
# Insert metadata
mycolunit.replace_one(
        {'userProfileId' : 'userProfile'},
        
        
        stats,
        upsert=True
    )

In [12]:
client.get_full_name()

'Adrien'